In [ ]:
import torch
torch.__version__

'2.1.0+cu118'

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:

from pprint import pprint
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import re

In [ ]:
base_url = 'https://skinsort.com/ingredients/'

In [ ]:
page = requests.get(url)

soup = BeautifulSoup(page.content, "html.parser")

In [ ]:
test = soup.find(id='ingredients-table')


bs4.element.Tag

In [ ]:


links = [a['href'] for a in test.find_all('a', href=True)]

In [ ]:
links


['/ingredients/niacinamide',
 '/ingredients/hyaluronic-acid',
 '/ingredients/salicylic-acid',
 '/ingredients/glycerin',
 '/ingredients/retinol',
 '/ingredients/ascorbic-acid',
 '/ingredients/water',
 '/ingredients/panthenol',
 '/ingredients/glycolic-acid',
 '/ingredients/centella-asiatica-extract',
 '/ingredients/azelaic-acid',
 '/ingredients/sodium-hyaluronate',
 '/ingredients/squalane',
 '/ingredients/lactic-acid',
 '/ingredients/tretinoin',
 '/ingredients/benzoyl-peroxide',
 '/ingredients/snail-secretion-filtrate',
 '/ingredients/bha',
 '/ingredients/ceramide-np',
 '/ingredients/ceramide-ap',
 '/ingredients/citric-acid',
 '/ingredients/tocopherol',
 '/ingredients/camellia-sinensis-leaf-extract',
 '/ingredients/alpha-arbutin',
 '/ingredients/page/2',
 '/ingredients/page/3',
 '#',
 '/ingredients/page/2',
 '/ingredients/page/1152']

In [ ]:

if test:
        # Extract links from the <a> (anchor) tags within the specified <div>
        links = [a['href'] for a in test.find_all('a')]


KeyError: ignored

In [ ]:
links

In [ ]:


def scrape_quotes(base_url, num_pages):
    page_links = []

    for page_num in tqdm(range(1, num_pages + 1)):
        #%debug
        url = f"{base_url}/page/{page_num}"
        response = requests.get(url)
        filtered_links = []
        links = []

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.find(id='ingredients-table')
            links = [a['href'] for a in table.find_all('a', href=True)]
            filtered_links = [link for link in links if "/ingredients/page" not in link and "#" not in link]
            page_links += filtered_links


        else:
            print(f"Failed to retrieve page {page_num}. Status code: {response.status_code}")

    return page_links




In [ ]:

num_pages = 1152  # Adjust the number of pages as needed

final_links = scrape_quotes(base_url, num_pages)


In [ ]:
df_links = pd.DataFrame(final_links)


,0
0,/ingredients/niacinamide
1,/ingredients/hyaluronic-acid
2,/ingredients/salicylic-acid
3,/ingredients/glycerin
4,/ingredients/retinol
...,...
28792,/ingredients/cannabis-sativa-sprout
28793,/ingredients/dibutyldecyl-ipdi
28794,/ingredients/capsella-bursa-pastoris-sprout-water
28795,/ingredients/carboxyethyl-acrylate


In [ ]:
#remove duplicates
df_links = df_links.drop_duplicates()
df_links

,0
0,/ingredients/niacinamide
1,/ingredients/hyaluronic-acid
2,/ingredients/salicylic-acid
3,/ingredients/glycerin
4,/ingredients/retinol
...,...
28791,/ingredients/caprae-lac-extract
28792,/ingredients/cannabis-sativa-sprout
28793,/ingredients/dibutyldecyl-ipdi
28794,/ingredients/capsella-bursa-pastoris-sprout-water


In [ ]:
data = pd.DataFrame(columns=['ingredients','explaination','reference'])

In [ ]:


def extract_text_from_class(url):
    df = pd.DataFrame(columns=['ingredients','explaination','reference'])
    base_url = 'https://skinsort.com'

    try:
        response = requests.get(base_url + url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        elements_with_class = soup.find_all(class_='mv-content ingredient-description')

        text_data = [element.get_text(strip=True) for element in elements_with_class][0][9:]

        elements_with_class_url = soup.find_all(class_='list-disc ml-5 mt-3')

        links = [element.get_text() for element in elements_with_class_url]

        # Extract links using regular expression
        links = re.findall(r'https?://\S+', ' '.join(text_with_links))







        df = df.append({'ingredients': url[13:], 'explaination': text_data, 'reference': links}, ignore_index=True)

        return df

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

if __name__ == "__main__":
    url = "/ingredients/niacinamide"  # Replace with the URL of the website you want to scrape
      # Replace with the actual class name you want to target

    text_data = extract_text_from_class(url)



text_data

,ingredients,explaination,reference
0,niacinamide,Niacinamide has emerged as an all-star ingredi...,[]


In [ ]:
for link in tqdm(df_links.iloc[:,0],desc="Processing links"):

    text_data = extract_text_from_class(link)
    data = data.append(text_data)

In [ ]:
data

,ingredients,explaination,reference
0,niacinamide,Niacinamide has emerged as an all-star ingredi...,"[https://pubmed.ncbi.nlm.nih.gov/17147561/, ht..."
0,hyaluronic-acid,Hyaluronic acid is naturally found in healthy ...,"[https://pubmed.ncbi.nlm.nih.gov/17147561/, ht..."
0,salicylic-acid,Salicylic Acid (also known asbeta hydroxy acid...,"[https://pubmed.ncbi.nlm.nih.gov/17147561/, ht..."
0,glycerin,Glycerin is already naturally found in your sk...,"[https://pubmed.ncbi.nlm.nih.gov/17147561/, ht..."
0,retinol,Retinol is a form of Vitamin A This anti-aging...,"[https://pubmed.ncbi.nlm.nih.gov/17147561/, ht..."
0,ascorbic-acid,Ascorbic Acid is is pure Vitamin C. Ascorbic A...,"[https://pubmed.ncbi.nlm.nih.gov/17147561/, ht..."
0,water,Water. It's the most common cosmetic ingredien...,"[https://pubmed.ncbi.nlm.nih.gov/17147561/, ht..."
0,panthenol,Panthenol (also referred to as pro-vitamin B5)...,"[https://pubmed.ncbi.nlm.nih.gov/17147561/, ht..."
0,glycolic-acid,Glycolic Acid is arguably the most famous AHA ...,"[https://pubmed.ncbi.nlm.nih.gov/17147561/, ht..."
0,centella-asiatica-extract,Centella Asiatica Extract is derived from an h...,"[https://pubmed.ncbi.nlm.nih.gov/17147561/, ht..."


In [ ]:
result_df

0       ingredients                                ...
1      ingredients                                 ...
2                                                 None
dtype: object

In [ ]:
import re

text_with_links = '\n\n\nhttps://pubmed.ncbi.nlm.nih.gov/17147561/\n\n\nhttps://www.komen.org/breast-cancer/survivorship/complementary-therapies/niacinamide/\n\n\nhttps://www.cosmeticsinfo.org/ingredients/niacinamide/\n\n\nhttps://pubmed.ncbi.nlm.nih.gov/24993939/\n\n\nhttps://www.ncbi.nlm.nih.gov/pmc/articles/PMC6824628/\n\n\nhttps://pubmed.ncbi.nlm.nih.gov/34844552/\n\n'

# Extract links using regular expression
links = re.findall(r'https?://\S+', text_with_links)

print(links)

['https://pubmed.ncbi.nlm.nih.gov/17147561/', 'https://www.komen.org/breast-cancer/survivorship/complementary-therapies/niacinamide/', 'https://www.cosmeticsinfo.org/ingredients/niacinamide/', 'https://pubmed.ncbi.nlm.nih.gov/24993939/', 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6824628/', 'https://pubmed.ncbi.nlm.nih.gov/34844552/']
